# SVM Implementation from scratch 

## Goals
In this guide, we’re going to implement the linear support vector machine algorithm from scratch in Python. Our goal will be to minimize the cost function, which we’ll use to train our model, and maximize the margin, which we’ll use to predict values against new, untrained data. We’ll be using NumPy — one of Python’s most popular machine learning libraries — to handle all of our calculations, so you won’t need any additional software or libraries installed on your computer to follow along!

## Mathematical Approach

### Overview 
SVM aims to maximize the gap between different groups. It wants points of one type on one side of a line and points of another type on the other. 

When this happens, points are sorted accurately based on how far they are from the line. A bigger gap means fewer mistakes in sorting. 

So, when the gap is largest, it's the best solution, making it easier to sort new points. If mistakes happen, regularization helps make better predictions for new points.

### Separating Hyperplane
A hyperplane is like a dividing line for data. It separates points into two groups. We use equations to find this line. In linear and logistic regression, it's y = mx + b. But for SVM, we need equations for both sides of the line, like in the SVM Diagram.

For the marginal plane, we can write the equation as:
$$w^T x+b=0$$

For the positive hyperplane the equation will be:

$$ w^T x+b \geq 0 \text { when } y_n=+1 $$

And for negative hyperplane:
$$ w^T x+b \geq 0 \text { when } y_n=-1 $$


### Soft margin SVM

using these hyperplanes equations we can derive the equation for the distance between positive and negative hyperplanes.

$$ w^T x_1+b-w^T x_2+b=1--1 $$
we will get
$$ w^T\left(x_1-x_2\right)=2 $$

Now we need to divide the equation with the norm of w to get the equation of distance :
$$
\begin{aligned}
& \frac{w^T}{\|w\|}\left(x_1-x_2\right)=\frac{2}{\|w\|} \\
& \mathrm{ie},\left(x_1-x_2\right)=\frac{2}{\|w\|}
\end{aligned}
$$

So here in order to maximize the marginal distance, we can maximize the equation we got subject to some constraints, ie, 
$$
\underset{w, b}\max \frac{2}{\|w\|}
$$
subject to
$$
y_n\left\{\begin{array}{cc}
+1 & w^T x+b \geq 1 \\
-1 & w^T x+b \leq-1
\end{array}\right\}
$$
which we can write as $y_n\left(w^T x+b\right) \geq 1$

This equation that needs to be maximized is known as the regularizer.
For performing optimization using gradient descent the regularizer can also be rewritten as follows:

$\begin{aligned} \max \frac{2}{\|w\|} & =\max \frac{1}{\|w\|} \\ =\min \|w\| & =\min \frac{1}{2}\|w\|^2\end{aligned}$

There are two more terminologies we need to include in this regularizer for formulating the optimization equation and those are the error terms including the number of errors in the training(C) and the sum of the value of error( Σζ ).
$$
\underbrace{\underset{w, b}\min \frac{1}{2}\|w\|^2}_{\text {regularizer }}+\underbrace{C_i \sum_{i=1}^n \xi_i}_{\text {error term }}
$$




### Loss function
In the case of Linear Regression, we used squared errors as the loss function, but in classification algorithms like SVM, it's not suitable to use squared errors. 
Is there any other loss function for SVM? Yes! it's the Hinge Loss function:
$$
f(x)=\max \{0,1-t\}
$$
By using this hinge loss function it gives an unconstrained optimization term:
$$
\underbrace{\underset{w, b}\min \frac{1}{2}\|w\|^2}_{\text {regularizer }}+\underbrace{C_i \sum_{i=1}^n \underset{w, b}\max \{0,1-y_n\left(w^T x+b\right)\}}_{\text {error term }}
$$

Using Gradient Descent we can find the best parameters w and b which we'll be implemented using python in the next section.

##  Implementing SVM from scratch in python


In [ ]:
import numpy as np

class SVM:

    def __init__(self, C = 1.0):
        # C = error term
        self.C = C
        self.w = 0
        self.b = 0

First, we created a class SVM and initialized some values. The C term is known as the error term which we need to add for optimizing the function

### Hinge Loss calculation

In [ ]:
# Hinge Loss Function / Calculation
def hingeloss(self, w, b, x, y):
    # Regularizer term
    reg = 0.5 * (w * w)

    for i in range(x.shape[0]):
        # Optimization term
        opt_term = y[i] * ((np.dot(w, x[i])) + b)

        # calculating loss
        loss = reg + self.C * max(0, 1-opt_term)
    return loss[0][0]

Let's understand what's happening here. First, we are calculating the value of the regularizer term and assign it to variable reg. Then iterating over the number of samples and calculating the loss using the optimization function.

### Gradient Descent optimization
Now let's create the gradient descent function inside the fit method in order to get the best parameters w and b. 


In [ ]:
def fit(self, X, Y, batch_size=100, learning_rate=0.001, epochs=1000):
    # The number of features in X
    number_of_features = X.shape[1]

    # The number of Samples in X
    number_of_samples = X.shape[0]

    c = self.C

    # Creating ids from 0 to number_of_samples - 1
    ids = np.arange(number_of_samples)

    # Shuffling the samples randomly
    np.random.shuffle(ids)

    # creating an array of zeros
    w = np.zeros((1, number_of_features))
    b = 0
    losses = []

    # Gradient Descent logic
    for i in range(epochs):
        # Calculating the Hinge Loss
        l = self.hingeloss(w, b, X, Y)

        # Appending all losses 
        losses.append(l)
        
        # Starting from 0 to the number of samples with batch_size as interval
        for batch_initial in range(0, number_of_samples, batch_size):
            gradw = 0
            gradb = 0

            for j in range(batch_initial, batch_initial + batch_size):
                if j < number_of_samples:
                    x = ids[j]
                    ti = Y[x] * (np.dot(w, X[x].T) + b)

                    if ti > 1:
                        gradw += 0
                        gradb += 0
                    else:
                        # Calculating the gradients

                        #w.r.t w 
                        gradw += c * Y[x] * X[x]
                        # w.r.t b
                        gradb += c * Y[x]

            # Updating weights and bias
            w = w - learning_rate * w + learning_rate * gradw
            b = b + learning_rate * gradb
    
    self.w = w
    self.b = b

    return self.w, self.b, losses 


What happens in this fit method is really simple, we are trying to reduce the loss in consecutive iterations and find the best parameters w and b. Note that here we are using Batch Gradient Descent. The weights(w) and bias(b) are updated in every iteration using the gradients and the learning rate resulting in the minimization of the loss. When the optimal parameters are found the method simply returns it along with the losses.

### Predict Method

In [ ]:
def predict(self, X):
        
        prediction = np.dot(X, self.w[0]) + self.b # w.x + b
        return np.sign(prediction)

### Performing Classification
Alright, we have created an SVM class only with the help of NumPy. Now let's do some classification to see our model in action. 

Creating random dataset

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from svm import SVM

# Creating dataset
X, y = datasets.make_blobs(

        n_samples = 100, # Number of samples
        n_features = 2, # Features
        centers = 2,
        cluster_std = 1,
        random_state=40
    )

# Classes 1 and -1
y = np.where(y == 0, -1, 1)

Splitting the dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

Training the SVM model

In [ ]:
svm = SVM()

w, b, losses = svm.fit(X_train, y_train)


Making predictions and testing accuracy

In [ ]:
prediction = svm.predict(X_test)

# Loss value
lss = losses.pop()

print("Loss:", lss)
print("Prediction:", prediction)
print("Accuracy:", accuracy_score(prediction, y_test))
print("w, b:", [w, b])

**Visualizing SVM**    
Visualizing SVM is one of the best ways to find how it is being fitted to the training data. We can do this using the matplotlib package.


In [ ]:
# Visualizing the scatter plot of the dataset
def visualize_dataset():
    plt.scatter(X[:, 0], X[:, 1], c=y)


# Visualizing SVM
def visualize_svm():

    def get_hyperplane_value(x, w, b, offset):
        return (-w[0][0] * x + b + offset) / w[0][1]

    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    plt.scatter(X_test[:, 0], X_test[:, 1], marker="o", c=y_test)

    x0_1 = np.amin(X_test[:, 0])
    x0_2 = np.amax(X_test[:, 0])

    x1_1 = get_hyperplane_value(x0_1, w, b, 0)
    x1_2 = get_hyperplane_value(x0_2, w, b, 0)

    x1_1_m = get_hyperplane_value(x0_1, w, b, -1)
    x1_2_m = get_hyperplane_value(x0_2, w, b, -1)

    x1_1_p = get_hyperplane_value(x0_1, w, b, 1)
    x1_2_p = get_hyperplane_value(x0_2, w, b, 1)

    ax.plot([x0_1, x0_2], [x1_1, x1_2], "y--")
    ax.plot([x0_1, x0_2], [x1_1_m, x1_2_m], "k")
    ax.plot([x0_1, x0_2], [x1_1_p, x1_2_p], "k")

    x1_min = np.amin(X[:, 1])
    x1_max = np.amax(X[:, 1])
    ax.set_ylim([x1_min - 3, x1_max + 3])

    plt.show()


visualize_dataset()
visualize_svm()